In [26]:
import jsonlines
import numpy as np
import pandas as pd

source_jsonl_ = '../data/raw/test_clean.jsonl'
chosen_cases_json = '../result/evaluation/chosen_sample_ids.json'

repo_dict_ = {}
original_repo_names_ = []
original_sample_ids_ = []

with jsonlines.open(source_jsonl_) as reader_:
    for obj_ in reader_:
        full_name_ = obj_['full_name']
        repo_dict_[full_name_] = repo_dict_.get(full_name_, 0) + 1
        original_repo_names_.append(full_name_)
        original_sample_ids_.append(obj_['sample_id'])

# Assuming 'data' is a DataFrame containing your dataset with 'repo_name' and 'case_count' columns
data = pd.DataFrame({
    'repo_name': list(repo_dict_.keys()),
    'case_count': list(repo_dict_.values())
})
original_dataset = pd.DataFrame({
    'repo_name': list(original_repo_names_),
    'sample_id': list(original_sample_ids_)
})

# Calculate the proportion of cases for each repository
data['proportion'] = data['case_count'] / data['case_count'].sum()

# Determine the number of cases to sample from each repository
data['sample_size'] = (data['proportion'] * 1000).round().astype(int)

# Adjust sample sizes to sum up to 1000
while data['sample_size'].sum() < 1000:
    diff = 1000 - data['sample_size'].sum()
    indices = data.index
    change_indices = np.random.choice(indices, abs(diff), replace=False)
    data.loc[change_indices, 'sample_size'] += np.sign(diff)

# Sample cases from each repository
data['sample_size'] = data.apply(lambda r: min(r['sample_size'], r['case_count']), axis=1)
selected_cases = pd.DataFrame()
for _, row in data.iterrows():
    repo_cases = original_dataset[original_dataset['repo_name'] == row['repo_name']]
    selected_cases = pd.concat([selected_cases, repo_cases.sample(n=row['sample_size'], replace=False)],
                               ignore_index=True)

# 'selected_cases' now contains 1000 cases fairly representing the whole dataset
selected_cases.to_json(chosen_cases_json, orient='records')


In [ ]:
generated_jsonl_ = '../result/candidates/candidates-20240407_154640.jsonl'
